In [1]:
import torch,sys,os
from torch import nn

from torch.utils.data import DataLoader
from torchvision import datasets

from torchvision.transforms import ToTensor,transforms

from tqdm import tqdm

In [2]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
# MNIST = datasets.MNIST('~/data/MNIST/')
# data_loader = torch.utils.data.DataLoader(MNIST,
#                                           batch_size=4,
#                                           shuffle=True,
#                                           num_workers=6)

In [4]:
from statistics import mode
from turtle import forward


class LeNet(nn.Module):
    def __init__(self, num_classes, init_weights=False) -> None:
        super().__init__()

        # self.conv1 = nn.Conv2d(in_channels=1,kernel_size=5,out_channels=6,stride=1)
        # self.maxPool = nn.MaxPool2d(kernel_size=2,stride=2)

        # self.conv2 = nn.Conv2d(in_channels=6,kernel_size=5,out_channels=16)

        # self.flattern = nn.Flatten()

        # self.fc1 = nn.Linear(in_features=120,out_features=84)
        # self.fc2 = nn.Linear(in_features=84,out_features=10)

        # self.out = nn.Softmax(dim=10)

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2))

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2))
        
        self.out = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=400,out_features=120),
            nn.Linear(in_features=120,out_features=84),
            nn.Linear(in_features=84,out_features=num_classes)
            )
            
        if init_weights:
            self.weight_init()

    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        y = self.out(x)
        return y

    def weight_init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='tanh')
                # nn.init.normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [5]:
model = LeNet(num_classes=10,init_weights=True).to(device)

In [6]:
print(model)

LeNet(
  (conv1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (out): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=400, out_features=120, bias=True)
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [7]:
# transform=transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5,0.5,0.5,),(0.5,0.5,0.5))
#     ]
# )

# Download training data from open datasets.
train_set = datasets.CIFAR10(
    root="~/data/CIFAR10/",
    train=True,
    download=True,
    transform=ToTensor() # transform,
)
trainloader=torch.utils.data.DataLoader(
	train_set,
	batch_size=36,
	shuffle=True,
	num_workers=0
	)


# Download test data from open datasets.
test_set = datasets.CIFAR10(
    root="~/data/CIFAR10/",
    train=False,
    download=True,
    transform=ToTensor() # transform,
)
testloader=torch.utils.data.DataLoader(
	test_set,
	batch_size=10000,
	shuffle=False,
	num_workers=0
	)

test_data_iter=iter(testloader)
test_image,test_label=test_data_iter.next()
test_num  = len(test_set)

train_steps = len(trainloader)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
# 定义一个损失函数
loss_fn = nn.CrossEntropyLoss()

# 定义一个优化器
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

epochs = 40

save_path= './LeNet.pth'
best_acc = 0.0


In [9]:
# train

for epoch in range(epochs):
        # train
        model.train()
        running_loss = 0.0
        train_bar = tqdm(trainloader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            outputs = model(images.to(device))
            loss = loss_fn(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)

        # validate
        model.eval()
        acc = 0.0  # accumulate accurate number / epoch
        with torch.no_grad():
            val_bar = tqdm(testloader, file=sys.stdout) # show progress
            for val_data in val_bar:
                val_images, val_labels = val_data
                outputs = model(val_images.to(device))
                predict_y = torch.max(outputs, dim=1)[1]
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

        val_accurate = acc / test_num
        print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, val_accurate))

        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(model.state_dict(), save_path)

print('Finished Training')

100%|██████████| 1/1 [00:01<00:00,  2.00s/it]
[epoch 1] train_loss: 1.874  val_accuracy: 0.366
100%|██████████| 1/1 [00:01<00:00,  1.12s/it]
[epoch 2] train_loss: 1.663  val_accuracy: 0.432
100%|██████████| 1/1 [00:01<00:00,  1.14s/it]
[epoch 3] train_loss: 1.546  val_accuracy: 0.444
100%|██████████| 1/1 [00:01<00:00,  1.11s/it]
[epoch 4] train_loss: 1.485  val_accuracy: 0.460
100%|██████████| 1/1 [00:01<00:00,  1.23s/it]
[epoch 5] train_loss: 1.449  val_accuracy: 0.493
100%|██████████| 1/1 [00:01<00:00,  1.13s/it]
[epoch 6] train_loss: 1.416  val_accuracy: 0.495
100%|██████████| 1/1 [00:01<00:00,  1.16s/it]
[epoch 7] train_loss: 1.394  val_accuracy: 0.502
100%|██████████| 1/1 [00:01<00:00,  1.18s/it]
[epoch 8] train_loss: 1.373  val_accuracy: 0.511
100%|██████████| 1/1 [00:01<00:00,  1.12s/it]
[epoch 9] train_loss: 1.353  val_accuracy: 0.511
100%|██████████| 1/1 [00:01<00:00,  1.12s/it]
[epoch 10] train_loss: 1.334  val_accuracy: 0.510
100%|██████████| 1/1 [00:01<00:00,  1.16s/it]
[ep

In [10]:
test_index = 5

test_lab = test_label[test_index]
test_img = val_images[test_index].unsqueeze(0)

weights_path = "./LeNet.pth"
assert os.path.exists(weights_path), "file: '{}' dose not exist.".format(weights_path)
model.load_state_dict(torch.load(weights_path))

model.eval()

LeNet(
  (conv1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (out): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=400, out_features=120, bias=True)
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [11]:

with torch.no_grad():
    outputs = model(test_img.to(device))
    predict_y = torch.max(outputs, dim=1)[1]
    print(predict_y)
    print(test_lab)

tensor([5], device='cuda:0')
tensor(6)
